In [70]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#We are going to pull in multiple urls many times so we will use a function to set urls and table names

def pull_lcs_stats(urlstats, urlroster):
    "urls should be strings.  This fuction pulls in player stats from one table and team info from another, putting them together and then cleaning the output and making it tidy"
    #package and send the request, store as r
    r = requests.get(urlstats).text

    #soup it
    soup = BeautifulSoup(r, features='lxml')
    #print(soup.prettify())

    #id the table
    MyTable = soup.find('table',{'class':'wikitable sortable spstats plainlinks hoverable-rows'})

    #string it for pandas
    soup_string = str(MyTable)

    #read it into pandas
    dfl = pd.read_html(soup_string)

    #create dataframe
    df1 = pd.DataFrame(dfl[0])
    
    #the teams column also used hyperlinking pictures, we need to merge from a second table.
    #package and send the request, store as r
    r2 = requests.get(urlroster).text

    #soup it
    soup2 = BeautifulSoup(r2, features='lxml')
    #print(soup.prettify())

    #id the table
    MyTable2 = soup2.find('table',{'class':'cargoTable sortable'})

    #string it for pandas
    soup_string2 = str(MyTable2)

    #read it into pandas
    dfr = pd.read_html(soup_string2)

    #create dataframe and take only the pertinent columns
    df2 = pd.DataFrame(dfr[0])
    df2 = df2.drop(columns= ['Role', 'Current Team', 'Role.1', 'Stream', 'Twitter', 'Birthday', 'Age'])
    
    #print(df2)
    
    #the data frame1 has bad column heads and organization, lets make a new one with just the value data.
    df = pd.DataFrame(df1.values, columns=['Team', 'ID', 'Games', 'Wins', 'Losses', 'Win Rate', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS', 'CS per Minute', 'Gold', 'Gold per Minute', 'Kill Part.', 'Kill Share', 'Gold Share', 'Champs Played', 'Champs'])
    #the champs column used pictures and we don't care much about that data, lets drop it
    df = df.drop(columns=['Team', 'Champs'])
    
    #print(df.info())
    df.ID = df.ID.str.lower()
    df2.ID = df2.ID.str.lower()
    
    #replace ID's based on a manual dictionary of pros that changed names at some point in their career {old name : current name}
    IDdict = {'brunch ü' : 'brunch', 'zionspartan' : 'darshan', 'i am anjo' : 'anjo', 'dontmashme' : 'mash', 'crumbzz' : 'crumbz', 'nientonsoh' : 'nien', 'reginald' : 'reginaid', 'sycho sid' : 'benny', 'mandatorycloud' : 'mancloud', 'balls' : 'balis', 'frommaplestreet' : 'maplestreet', 'evaniskus' : 'nisker', 'krepo' : 'boris', 'wizfujiin':'apollo', 'iwdominate':'iwilldominate', 'bunnyfufuu':'bunny fufuu', 'baby':'big', 'ackerman':'godlike', 'dodo8':'dodo', 'konkwon':'kwon', 'imagine':'yolo', 'flaresz':'aetheres', 'incarnati0n':'jensen', 'trashy':'kold', 'lod':'benji', 'alexich':'alex ich', 'remi':'remilia', 'grigne':'grig', 'papa chau':'papachau'}
    df = df.replace({'ID' : IDdict})
    
    #random typo issue with one source
    bugdict = {'[[file:unrecognized country!.png|unrecognized country!|link=]] corejj':'corejj'}
    df2 = df2.replace({'ID' : bugdict})
    
    #take the team data from 2 and put it in dff
    dff = pd.merge(df, df2, on='ID', validate = "1:m")
    
    #get rid of non-numerics in certain columns and convert strings to numeric turning missing data to NaN
    dff['Win Rate'] = dff['Win Rate'].str.replace('%', '')
    dff['Gold'] = dff['Gold'].str.replace('k', '')
    dff['Kill Part.'] = dff['Kill Part.'].str.replace('%', '')
    dff['Kill Share'] = dff['Kill Share'].str.replace('%', '')
    dff['Gold Share'] = dff['Gold Share'].str.replace('%', '')
    
    for i in range(1,17):
        dff[dff.columns[i]] = pd.to_numeric(dff[dff.columns[i]], errors='coerce')
    
    
    return dff


In [71]:
def winners13(url):
    "the url has to be a string.  This function pulls a table of team ranks for both spring and summer playoffs"
    #package and send the request, store as r
    r = requests.get(url).text

    #soup it
    soup = BeautifulSoup(r, features='lxml')
    #print(soup.prettify())

    #id the table
    MyTable = soup.find('table',{'class':'wikitable zebra'})

    #string it for pandas
    soup_string = str(MyTable)

    #read it into pandas
    dfl = pd.read_html(soup_string)

    #create dataframe
    dft = pd.DataFrame(dfl[0])
    df = pd.DataFrame(dft.values, columns=['Place', 'Prize', 'Spring', 'Summer'])
    df = df.drop(0)
    df = df.drop(1)
    df = df.drop(columns='Prize')
    
    
    #replace Team names based on a manual dictionary {old name : current name}
    IDdict = {'Counter Logic Gaming':'CLG'}
    df = df.replace({'Spring' : IDdict})
    df = df.replace({'Summer' : IDdict})
    
    #make placement numeric for later use
    df['Place'] = [1, 2, 3, 4, 5, 5, 7, 7]
    
    return df

def winners14(url):
    "the url has to be a string.  This function pulls a table of team ranks for both spring and summer playoffs"
    #package and send the request, store as r
    r = requests.get(url).text

    #soup it
    soup = BeautifulSoup(r, features='lxml')
    #print(soup.prettify())

    #id the table
    MyTable = soup.find('table',{'class':'wikitable zebra'})

    #string it for pandas
    soup_string = str(MyTable)

    #read it into pandas
    dfl = pd.read_html(soup_string)

    #create dataframe
    dft = pd.DataFrame(dfl[0])
    df = pd.DataFrame(dft.values, columns=['Place', 'Prize', 'Spring', 'Summer'])
    df = df.drop(0)
    df = df.drop(1)
    df = df.drop(columns='Prize')
    
    
    #replace Team names based on a manual dictionary {old name : current name}
    IDdict = {'Counter Logic Gaming':'CLG'}
    df = df.replace({'Spring' : IDdict})
    df = df.replace({'Summer' : IDdict})
    
    #make placement numeric for later use
    df['Place'] = [1, 2, 3, 4, 5, 6, 7, 7]
    
    return df

def winners15(url):
    "the url has to be a string.  This function pulls a table of team ranks for both spring and summer playoffs"
    #package and send the request, store as r
    r = requests.get(url).text

    #soup it
    soup = BeautifulSoup(r, features='lxml')
    #print(soup.prettify())

    #id the table
    MyTable = soup.find('table',{'class':'wikitable2 tournament-results zebra-manual'})

    #string it for pandas
    soup_string = str(MyTable)

    #read it into pandas
    dfl = pd.read_html(soup_string)

    #create dataframe
    df = pd.DataFrame(dfl[0])
        
    #replace Team names based on a manual dictionary {old name : current name}
    IDdict = {'Counter Logic Gaming':'CLG'}
    df = df.replace({'Team' : IDdict})
    
    #sometimes they randomly change the names of columns... no consistency!
    df = df.rename(columns={'CP':'Championship Points', 'Points':'Championship Points'})
    
    #drop unwanted columns
    df = df.drop(columns=['Prize', 'Prize (%)', 'Championship Points', 'Qualification'])
    
    #make placement numeric for later use
    df['Place'] = [1, 2, 3, 4, 5, 5]
    
    return df

In [72]:
#phew, ok, now we have tidy data that has been fairly well checked for errors and missing values
#But it's in dozens of dataframes.  Let's work on melting them with year and season data so we can combine into one
#homestretch!

#let's make a function that will automate this
def seasontag(dfd, split, kind, year):
    "adds spring/summer, season/playoff, and year to dataframe.  Enter info as (dataframe, split, kind, year)"
    #first, let's make new columns with entries for these
    dfd['Year'] = year
    dfd['Split'] = split.lower()
    dfd['Kind'] = kind.lower()
    
    #lets reorder so everything starts with year, split, kind, team, id, name, place, then etc.
    cols = dfd.columns.tolist()
    cols = cols[-3:] + cols[-6:-5] + cols[0:1] + cols[-5:-3] + cols[1:-6]

    dfd = dfd[cols]
    
    return dfd


In [73]:
spring_split_2013 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202013%20Spring&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/Season%203/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#spring2013 has a duplicate for bloodwater but he played no games on GGU so drop the first entry and leave the second
spring_split_2013 = spring_split_2013.drop(48)
#check that the second and third .info() prints have the same rows
#print(spring_split_2013.info())


In [74]:
spring_ranks = winners13('https://lol.gamepedia.com/NA_LCS/Season_3')
spring_ranks = spring_ranks.drop(columns='Summer')

spring_split_2013 = pd.merge(left=spring_split_2013, right=spring_ranks, left_on='Team', right_on='Spring', validate='m:1')
spring_split_2013 = spring_split_2013.drop(columns='Spring')
print(spring_split_2013)

              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0         brunch     28     9      19      32.1   3.46    3.21     6.75  3.18   
1    lautemortis     28     9      19      32.1   1.43    3.93     8.68  2.57   
2          meyea     28     9      19      32.1   0.96    3.64     9.93  2.99   
3         nickwu     28     9      19      32.1   4.64    3.61     5.86  2.91   
4        chuuper     18     4      14      22.2   3.61    3.78     5.33  2.37   
5         pr0lly     10     5       5      50.0   4.30    3.50     7.60  3.40   
6       aphromoo     28    13      15      46.4   0.71    2.46     9.39  4.10   
7     doublelift     28    13      15      46.4   5.43    2.68     4.29  3.63   
8      hotshotgg     28    13      15      46.4   1.86    2.93     6.29  2.78   
9           link     28    13      15      46.4   3.21    2.29     6.46  4.23   
10      chauster     26    12      14      46.2   0.96    3.27     8.15  2.79   
11      bigfatlp      2     

In [75]:
spring_playoffs_2013 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202013%20Spring%20Playoffs&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/Season%203/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2013.info())

spring_ranks = winners13('https://lol.gamepedia.com/NA_LCS/Season_3')
spring_ranks = spring_ranks.drop(columns='Summer')

spring_playoffs_2013 = pd.merge(left=spring_playoffs_2013, right=spring_ranks, left_on='Team', right_on='Spring', validate='m:1')
spring_playoffs_2013 = spring_playoffs_2013.drop(columns='Spring')
print(spring_playoffs_2013)

              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0       aphromoo      3     1       2      33.3   1.00    4.00     7.00   
1       chauster      3     1       2      33.3   1.00    2.67     6.33   
2     doublelift      3     1       2      33.3   1.00    3.67     4.33   
3      hotshotgg      3     1       2      33.3   2.33    2.33     5.33   
4           link      3     1       2      33.3   3.67    1.33     4.33   
5        darshan     11     6       5      54.5   3.55    1.64     5.55   
6     daydreamin     11     6       5      54.5   0.82    3.36     7.91   
7         jintae     11     6       5      54.5   2.27    3.55     7.18   
8           mash     11     6       5      54.5   4.64    2.27     4.55   
9    nintendudex     11     6       5      54.5   1.45    2.82     7.91   
10           cop      6     2       4      33.3   6.67    2.17     6.83   
11       nyjacky      6     2       4      33.3   4.50    4.67     9.17   
12          rhux      6  

In [76]:
summer_split_2013 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202013%20Summer&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/Season%203/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_split_2013.info())

summer_ranks = winners13('https://lol.gamepedia.com/NA_LCS/Season_3')
summer_ranks = summer_ranks.drop(columns='Spring')

summer_split_2013 = pd.merge(left=summer_split_2013, right=summer_ranks, left_on='Team', right_on='Summer', validate='m:1')
summer_split_2013 = summer_split_2013.drop(columns='Summer')
print(summer_split_2013)

              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0          balis     28    25       3      89.3   4.32    2.89     7.54   
1            hai     28    25       3      89.3   4.86    2.71     7.93   
2    lemonnation     28    25       3      89.3   1.14    2.86    11.14   
3         meteos     28    25       3      89.3   2.96    1.04    10.14   
4         sneaky     28    25       3      89.3   4.71    1.75     7.46   
5       bigfatlp     29    13      16      44.8   1.14    3.10     8.31   
6       chauster     29    13      16      44.8   0.62    3.79     8.14   
7     doublelift     29    13      16      44.8   4.76    2.93     5.17   
8           link     29    13      16      44.8   4.48    3.21     6.00   
9           nien     29    13      16      44.8   3.31    3.00     6.21   
10    daydreamin     28     9      19      32.1   0.86    3.18     6.75   
11          mash     28     9      19      32.1   2.75    2.25     5.18   
12   nintendudex     28  

In [77]:
summer_playoffs_2013 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202013%20Summer%20Playoffs&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/Season%203/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2013.info())

summer_ranks = winners13('https://lol.gamepedia.com/NA_LCS/Season_3')
summer_ranks = summer_ranks.drop(columns='Spring')

summer_playoffs_2013 = pd.merge(left=summer_playoffs_2013, right=summer_ranks, left_on='Team', right_on='Summer', validate='m:1')
summer_playoffs_2013 = summer_playoffs_2013.drop(columns='Summer')
print(summer_playoffs_2013)

              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0          balis      5     5       0     100.0   3.80    1.80     8.80   
1            hai      5     5       0     100.0   6.00    1.60     8.80   
2    lemonnation      5     5       0     100.0   1.20    2.00    13.60   
3         meteos      5     5       0     100.0   4.80    1.00    10.20   
4         sneaky      5     5       0     100.0   6.00    1.80     7.60   
5       bigfatlp      4     2       2      50.0   2.75    2.00     7.50   
6       chauster      4     2       2      50.0   0.75    2.75     8.50   
7     doublelift      4     2       2      50.0   4.50    1.75     4.75   
8           link      4     2       2      50.0   3.00    2.50     6.75   
9           nien      4     2       2      50.0   3.25    2.75     6.25   
10           cop      5     1       4      20.0   1.00    2.60     5.60   
11        edward      5     1       4      20.0   1.40    2.40     5.00   
12       nyjacky      5  

In [78]:
spring_split_2014 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202014%20Spring&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics','https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2014%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_split_2014.info())

spring_ranks = winners14('https://lol.gamepedia.com/NA_LCS/2014_Season')
spring_ranks = spring_ranks.drop(columns='Summer')

spring_split_2014 = pd.merge(left=spring_split_2014, right=spring_ranks, left_on='Team', right_on='Spring', validate='m:1')
spring_split_2014 = spring_split_2014.drop(columns='Spring')
print(spring_split_2014)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis     28    24       4      85.7   3.00    1.64     7.50   
1             hai     28    24       4      85.7   4.71    3.00     7.14   
2     lemonnation     28    24       4      85.7   1.04    2.75    11.18   
3          meteos     28    24       4      85.7   3.82    1.68     8.79   
4          sneaky     28    24       4      85.7   5.11    1.57     7.00   
5        aphromoo     28    18      10      64.3   1.14    2.50     9.61   
6      doublelift     28    18      10      64.3   4.32    2.21     6.04   
7            link     28    18      10      64.3   3.68    2.32     6.18   
8            nien     28    18      10      64.3   2.36    2.57     5.96   
9          dexter     19    14       5      73.7   3.74    2.58     6.95   
10      hotshotgg      6     2       4      33.3   0.33    3.00     4.17   
11       chauster      3     2       1      66.7   2.33    2.00     6.00   
12          

In [79]:
spring_playoffs_2014 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202014%20Spring%20Playoffs&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics','https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2014%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2014.info())

spring_ranks = winners14('https://lol.gamepedia.com/NA_LCS/2014_Season')
spring_ranks = spring_ranks.drop(columns='Summer')

spring_playoffs_2014 = pd.merge(left=spring_playoffs_2014, right=spring_ranks, left_on='Team', right_on='Spring', validate='m:1')
spring_playoffs_2014 = spring_playoffs_2014.drop(columns='Spring')
print(spring_playoffs_2014)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis      5     5       0     100.0   4.00    2.00     8.20   
1             hai      5     5       0     100.0   3.80    0.40    10.00   
2     lemonnation      5     5       0     100.0   0.00    1.60    13.80   
3          meteos      5     5       0     100.0   5.00    0.60     8.20   
4          sneaky      5     5       0     100.0   5.00    0.80     8.80   
5        aphromoo      7     5       2      71.4   0.29    1.86     8.71   
6          dexter      7     5       2      71.4   2.57    2.43     4.71   
7      doublelift      7     5       2      71.4   3.86    1.43     5.14   
8            link      7     5       2      71.4   2.57    1.71     6.71   
9            nien      7     5       2      71.4   2.29    1.71     6.29   
10         apollo      5     1       4      20.0   2.40    1.80     4.20   
11        darshan      5     1       4      20.0   2.60    2.80     3.20   
12     daydr

In [80]:
summer_split_2014 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202014%20Summer&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics','https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2014%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_split_2014.info())

summer_ranks = winners14('https://lol.gamepedia.com/NA_LCS/2014_Season')
summer_ranks = summer_ranks.drop(columns='Spring')

summer_split_2014 = pd.merge(left=summer_split_2014, right=summer_ranks, left_on='Team', right_on='Summer', validate='m:1')
summer_split_2014 = summer_split_2014.drop(columns='Summer')
print(summer_split_2014)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis     28    18      10      64.3   2.00    1.93     4.71   
1             hai     28    18      10      64.3   3.50    3.00     5.50   
2     lemonnation     28    18      10      64.3   0.71    2.61     8.00   
3          meteos     28    18      10      64.3   1.89    1.82     6.14   
4          sneaky     28    18      10      64.3   4.04    1.54     4.54   
5        bubbadub     28    10      18      35.7   0.64    3.07     6.11   
6      robertxlee     28    10      18      35.7   3.00    2.82     3.68   
7        westrice     28    10      18      35.7   1.89    3.64     4.36   
8          pr0lly     26    10      16      38.5   2.38    3.54     4.12   
9             kez     22     9      13      40.9   1.86    2.45     5.36   
10    brokenshard      6     1       5      16.7   1.50    4.17     3.17   
11       mancloud      2     0       2       0.0   1.50    1.50     1.50   
12       aph

In [81]:
summer_playoffs_2014 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=NA%20LCS%202014%20Summer%20Playoffs&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2014%20Season/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2014.info())

summer_ranks = winners14('https://lol.gamepedia.com/NA_LCS/2014_Season')
summer_ranks = summer_ranks.drop(columns='Spring')

summer_playoffs_2014 = pd.merge(left=summer_playoffs_2014, right=summer_ranks, left_on='Team', right_on='Summer', validate='m:1')
summer_playoffs_2014 = summer_playoffs_2014.drop(columns='Summer')
print(summer_playoffs_2014)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis      8     5       3      62.5   1.50    1.50     5.88   
1             hai      8     5       3      62.5   4.00    3.00     5.50   
2     lemonnation      8     5       3      62.5   0.50    1.50     9.00   
3          meteos      8     5       3      62.5   2.38    1.13     5.50   
4          sneaky      8     5       3      62.5   4.25    1.00     3.88   
5        aphromoo      7     1       6      14.3   0.71    3.14     6.00   
6          dexter      7     1       6      14.3   1.14    2.71     5.71   
7      doublelift      7     1       6      14.3   4.43    2.14     3.14   
8            link      7     1       6      14.3   2.43    2.71     3.71   
9          seraph      7     1       6      14.3   1.00    3.29     5.00   
10        godlike     10     5       5      50.0   1.90    3.60     9.10   
11            mor     10     5       5      50.0   0.80    3.10    10.70   
12         n

In [82]:
spring_split_2015 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202015%20Spring&TS%5Bshowstats%5D=2015&pfRunQueryFormName=TournamentStatistics','https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2015%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_split_2015.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2015_Season/Spring_Playoffs')

spring_split_2015 = pd.merge(left=spring_split_2015, right=spring_ranks, on='Team', how='left')
spring_split_2015.fillna(7, inplace=True)
spring_split_2015.Place = spring_split_2015.Place.astype(int)
print(spring_split_2015)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis     19    13       6      68.4   1.63    2.37     6.47   
1             hai     19    13       6      68.4   2.89    2.21     5.95   
2     lemonnation     19    13       6      68.4   1.05    2.00     8.16   
3          meteos     19    13       6      68.4   2.00    1.58     7.42   
4          sneaky     19    13       6      68.4   4.68    1.32     4.74   
5        aphromoo     19    12       7      63.2   0.89    2.32     9.68   
6      doublelift     19    12       7      63.2   3.95    1.95     5.74   
7            link     19    12       7      63.2   4.79    2.26     5.47   
8         xmithie     19    12       7      63.2   1.74    2.53     7.63   
9         darshan     17    11       6      64.7   2.18    2.06     6.76   
10          benny      2     1       1      50.0   3.00    3.00    10.00   
11    bunny fufuu     18    10       8      55.6   0.83    2.06     9.33   
12          

In [83]:
spring_playoffs_2015 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202015%20Spring%20Playoffs&TS%5Bshowstats%5D=2015&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2015%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2015.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2015_Season/Spring_Playoffs')

spring_playoffs_2015 = pd.merge(left=spring_playoffs_2015, right=spring_ranks, on='Team', how='left')
spring_playoffs_2015.fillna(7, inplace=True)
spring_playoffs_2015.Place = spring_playoffs_2015.Place.astype(int)
print(spring_playoffs_2015)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis      9     4       5      44.4   2.00    2.44     4.89   
1             hai      9     4       5      44.4   2.44    3.33     3.89   
2     lemonnation      9     4       5      44.4   0.56    3.22     7.11   
3          meteos      9     4       5      44.4   0.67    2.11     6.89   
4          sneaky      9     4       5      44.4   4.00    2.11     4.00   
5        aphromoo      3     0       3       0.0   1.33    4.67     4.67   
6         darshan      3     0       3       0.0   1.67    4.33     3.67   
7      doublelift      3     0       3       0.0   2.67    3.67     3.33   
8            link      3     0       3       0.0   2.00    5.00     4.33   
9         xmithie      3     0       3       0.0   1.33    5.33     5.67   
10    bunny fufuu      4     1       3      25.0   0.25    3.50     5.75   
11            cop      4     1       3      25.0   3.00    3.00     3.75   
12       hau

In [84]:
summer_split_2015 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202015%20Summer&TS%5Bshowstats%5D=2015&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2015%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_split_2015.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2015_Season/Summer_Playoffs')

summer_split_2015 = pd.merge(left=summer_split_2015, right=summer_ranks, on='Team', how='left')
summer_split_2015.fillna(7, inplace=True)
summer_split_2015.Place = summer_split_2015.Place.astype(int)
print(summer_split_2015)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           balis     19     7      12      36.8   2.16    3.53     6.58   
1          jensen     19     7      12      36.8   3.42    2.58     5.05   
2     lemonnation     19     7      12      36.8   0.63    3.53     8.47   
3          sneaky     19     7      12      36.8   4.21    2.37     5.00   
4          meteos     10     3       7      30.0   1.10    2.50     6.50   
5             hai      9     4       5      44.4   2.44    2.67     6.33   
6        aphromoo     19    13       6      68.4   1.00    2.89    11.84   
7         darshan     19    13       6      68.4   3.84    2.53     6.68   
8      doublelift     19    13       6      68.4   6.16    2.58     6.84   
9         xmithie     19    13       6      68.4   1.58    2.21     9.16   
10       aetheres     18     4      14      22.2   2.06    5.00     4.50   
11       bodydrop     18     4      14      22.2   0.61    4.17     6.61   
12          

In [85]:
summer_playoffs_2015 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202015%20Summer%20Playoffs&TS%5Bshowstats%5D=2015&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2015%20Season/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2015.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2015_Season/Summer_Playoffs')

summer_playoffs_2015 = pd.merge(left=summer_playoffs_2015, right=summer_ranks, on='Team', how='left')
summer_playoffs_2015.fillna(7, inplace=True)
summer_playoffs_2015.Place = summer_playoffs_2015.Place.astype(int)
print(summer_playoffs_2015)

               ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0        aphromoo      6     6       0     100.0   0.50    2.67    15.50   
1         darshan      6     6       0     100.0   3.50    1.67     9.33   
2      doublelift      6     6       0     100.0   7.17    2.00     9.33   
3         xmithie      6     6       0     100.0   2.83    2.00    12.83   
4           altec      4     1       3      25.0   3.00    1.25     3.25   
5     bunny fufuu      4     1       3      25.0   0.50    2.00     5.00   
6        hauntzer      4     1       3      25.0   1.00    2.00     5.50   
7            move      4     1       3      25.0   1.25    3.50     4.00   
8          corejj      3     0       3       0.0   4.67    3.00     2.33   
9           gamsu      3     0       3       0.0   1.33    2.00     4.33   
10         helios      3     0       3       0.0   1.00    2.00     6.00   
11        kiwikid      3     0       3       0.0   0.00    4.00     5.33   
12       shi

In [86]:
spring_split_2016 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202016%20Spring&TS%5Bshowstats%5D=2016&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2016%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#several players were traded around but never played for multiple teams in stats, drop those entries
spring_split_2016 = spring_split_2016.drop(40)
spring_split_2016 = spring_split_2016.drop(30)
spring_split_2016 = spring_split_2016.drop(32)
#check that the second and third .info() prints have the same rows
#print(spring_split_2016.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2016_Season/Spring_Playoffs')

spring_split_2016 = pd.merge(left=spring_split_2016, right=spring_ranks, on='Team', how='left')
spring_split_2016.fillna(7, inplace=True)
spring_split_2016.Place = spring_split_2016.Place.astype(int)
print(spring_split_2016)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0        balis     18    12       6      66.7   2.22    2.39     5.39  3.19   
1       jensen     18    12       6      66.7   4.50    2.28     5.39  4.34   
2         rush     18    12       6      66.7   2.50    2.11     7.50  4.74   
3       sneaky     18    12       6      66.7   4.44    2.39     5.28  4.07   
4          hai     16    12       4      75.0   0.50    2.69    10.81  4.21   
..         ...    ...   ...     ...       ...    ...     ...      ...   ...   
57    smoothie      1     0       1       0.0   0.00    4.00     9.00  2.25   
58    bjergsen     18     9       9      50.0   3.06    2.11     6.06  4.32   
59  doublelift     18     9       9      50.0   4.61    2.61     4.28  3.40   
60    hauntzer     18     9       9      50.0   2.28    2.39     4.94  3.02   
61  yellowstar     18     9       9      50.0   0.11    3.00     8.72  2.94   

        CS  CS per Minute  Gold  Gold per Minute  K

In [87]:
spring_playoffs_2016 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202016%20Spring%20Playoffs&TS%5Bshowstats%5D=2016&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2016%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2016.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2016_Season/Spring_Playoffs')

spring_playoffs_2016 = pd.merge(left=spring_playoffs_2016, right=spring_ranks, on='Team', how='left')
spring_playoffs_2016.fillna(7, inplace=True)
spring_playoffs_2016.Place = spring_playoffs_2016.Place.astype(int)
print(spring_playoffs_2016)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists    KDA  \
0        balis      4     1       3      25.0   0.50    3.75     4.25   1.27   
1          hai      4     1       3      25.0   0.00    4.25     5.25   1.24   
2       jensen      4     1       3      25.0   3.75    2.50     3.75   3.00   
3         rush      4     1       3      25.0   2.00    5.25     4.50   1.24   
4       sneaky      4     1       3      25.0   3.25    2.25     4.00   3.22   
5     aphromoo     10     6       4      60.0   0.20    2.90     8.10   2.86   
6      darshan     10     6       4      60.0   3.40    2.40     6.40   4.08   
7         huhi     10     6       4      60.0   2.50    2.50     6.50   3.60   
8      xmithie     10     6       4      60.0   1.90    2.80     6.70   3.07   
9       adrian      6     3       3      50.0   0.83    2.50    10.17   4.40   
10        huni      6     3       3      50.0   2.50    3.33     5.17   2.30   
11   reignover      6     3       3     

In [88]:
summer_split_2016 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202016%20Summer&TS%5Bshowstats%5D=2016&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2016%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_split_2016.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2016_Season/Summer_Playoffs')

summer_split_2016 = pd.merge(left=summer_split_2016, right=summer_ranks, on='Team', how='left')
summer_split_2016.fillna(7, inplace=True)
summer_split_2016.Place = summer_split_2016.Place.astype(int)
print(summer_split_2016)


             ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0        apollo     46    21      25      45.7   2.59    2.07     5.91  4.12   
1           ray     46    21      25      45.7   3.43    3.20     5.52  2.80   
2        shrimp     46    21      25      45.7   2.11    2.85     6.54  3.04   
3       xpecial     46    21      25      45.7   0.50    2.76     8.63  3.31   
4        impact     44    28      16      63.6   2.32    2.23     6.48  3.95   
5        jensen     44    28      16      63.6   4.70    2.05     5.05  4.77   
6        meteos     44    28      16      63.6   2.05    1.43     7.48  6.65   
7        sneaky     44    28      16      63.6   4.25    1.64     6.09  6.32   
8      smoothie     28    19       9      67.9   0.46    1.86     9.96  5.62   
9   bunny fufuu     16     9       7      56.3   0.44    1.69     8.81  5.48   
10     aphromoo     46    25      21      54.3   0.61    2.54     7.46  3.17   
11      darshan     46    25      21    

In [89]:
summer_playoffs_2016 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202016%20Summer%20Playoffs&TS%5Bshowstats%5D=2016&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2016%20Season/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2016.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2016_Season/Summer_Playoffs')

summer_playoffs_2016 = pd.merge(left=summer_playoffs_2016, right=summer_ranks, on='Team', how='left')
summer_playoffs_2016.fillna(7, inplace=True)
summer_playoffs_2016.Place = summer_playoffs_2016.Place.astype(int)
print(summer_playoffs_2016)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0       impact     13     7       6      53.8   3.23    3.08     7.38  3.45   
1       jensen     13     7       6      53.8   6.54    2.15     5.54  5.61   
2       meteos     13     7       6      53.8   2.46    2.23     7.85  4.62   
3     smoothie     13     7       6      53.8   0.46    2.62    10.38  4.15   
4       sneaky     13     7       6      53.8   3.38    1.85     8.54  6.46   
5     aphromoo     12     5       7      41.7   0.33    2.67     9.92  3.84   
6      darshan     12     5       7      41.7   1.92    2.25     6.50  3.74   
7         huhi     12     5       7      41.7   3.75    2.83     4.92  3.06   
8      xmithie     12     5       7      41.7   2.08    2.50     7.75  3.93   
9       adrian     10     5       5      50.0   0.40    2.70     9.70  3.74   
10        huni     10     5       5      50.0   3.70    4.00     6.10  2.45   
11   reignover     10     5       5      50.0   2.80

In [90]:
spring_split_2017 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202017%20Spring&TS%5Bshowstats%5D=2017&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2017%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#drop two dups
spring_split_2017 = spring_split_2017.drop(33)
spring_split_2017 = spring_split_2017.drop(50)
#check that the second and third .info() prints have the same rows
#print(spring_split_2017.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2017_Season/Spring_Playoffs')

spring_split_2017 = pd.merge(left=spring_split_2017, right=spring_ranks, on='Team', how='left')
spring_split_2017.fillna(7, inplace=True)
spring_split_2017.Place = spring_split_2017.Place.astype(int)
print(spring_split_2017)


             ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0     contractz     44    31      13      70.5   3.64    2.82     6.73  3.68   
1        jensen     44    31      13      70.5   4.89    2.00     5.93  5.41   
2      smoothie     44    31      13      70.5   1.16    1.93     9.32  5.42   
3        sneaky     44    31      13      70.5   3.27    2.30     6.39  4.21   
4        impact     34    25       9      73.5   2.29    2.35     7.71  4.25   
5           ray     10     6       4      60.0   2.80    3.50     5.60  2.40   
6      aphromoo     44    22      22      50.0   0.73    3.64     8.73  2.60   
7       darshan     44    22      22      50.0   2.70    3.16     6.34  2.86   
8          huhi     44    22      22      50.0   4.25    3.41     5.41  2.83   
9       xmithie     44    22      22      50.0   3.02    3.73     6.09  2.45   
10         joey      2     0       2       0.0   0.00    5.50     4.00  0.73   
11     akaadian     43    15      28    

In [91]:
spring_playoffs_2017 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202017%20Spring%20Playoffs&TS%5Bshowstats%5D=2017&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2017%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2017.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2017_Season/Spring_Playoffs')

spring_playoffs_2017 = pd.merge(left=spring_playoffs_2017, right=spring_ranks, on='Team', how='left')
spring_playoffs_2017.fillna(7, inplace=True)
spring_playoffs_2017.Place = spring_playoffs_2017.Place.astype(int)
print(spring_playoffs_2017)


             ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0     contractz      8     5       3      62.5   3.50    2.13     4.38  3.71   
1        jensen      8     5       3      62.5   3.25    1.88     3.50  3.60   
2      smoothie      8     5       3      62.5   0.63    1.50     6.50  4.75   
3        sneaky      8     5       3      62.5   2.50    2.00     4.50  3.50   
4        impact      4     3       1      75.0   1.00    1.50     5.50  4.33   
5           ray      4     2       2      50.0   2.75    4.75     5.50  1.74   
6      aphromoo      5     2       3      40.0   0.40    2.00     9.00  4.70   
7       darshan      5     2       3      40.0   3.60    2.20     4.40  3.64   
8          huhi      5     2       3      40.0   4.80    4.20     4.80  2.29   
9       xmithie      5     2       3      40.0   2.00    2.40     6.00  3.33   
10        altec     13     5       8      38.5   2.92    2.46     3.85  2.75   
11        balis     13     5       8    

In [92]:
summer_split_2017 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202017%20Summer&TS%5Bshowstats%5D=2017&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2017%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#3 dups to remove
summer_split_2017 = summer_split_2017.drop(36)
summer_split_2017 = summer_split_2017.drop(55)
summer_split_2017 = summer_split_2017.drop(60)
#check that the second and third .info() prints have the same rows
#print(summer_split_2017.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2017_Season/Summer_Playoffs')

summer_split_2017 = pd.merge(left=summer_split_2017, right=summer_ranks, on='Team', how='left')
summer_split_2017.fillna(7, inplace=True)
summer_split_2017.Place = summer_split_2017.Place.astype(int)
print(summer_split_2017)


             ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0     contractz     45    28      17      62.2   2.60    2.44     7.40  4.09   
1        jensen     45    28      17      62.2   5.69    1.27     5.33  8.70   
2      smoothie     45    28      17      62.2   0.87    2.13    10.02  5.10   
3        sneaky     45    28      17      62.2   3.24    1.96     6.58  5.02   
4        impact     31    21      10      67.7   1.68    2.58     6.90  3.33   
5           ray     14     7       7      50.0   1.71    2.79     6.43  2.92   
6      aphromoo     46    27      19      58.7   0.65    2.98     9.09  3.27   
7       darshan     46    27      19      58.7   3.57    2.20     5.67  4.21   
8          huhi     46    27      19      58.7   4.54    2.28     6.11  4.67   
9       omargod     14     7       7      50.0   2.00    2.64     8.43  3.95   
10         gate     43    16      27      37.2   0.65    2.60     5.63  2.41   
11      froggen     38    15      23    

In [93]:
summer_playoffs_2017 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202017%20Summer%20Playoffs&TS%5Bshowstats%5D=2017&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2017%20Season/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2017.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2017_Season/Summer_Playoffs')

summer_playoffs_2017 = pd.merge(left=summer_playoffs_2017, right=summer_ranks, on='Team', how='left')
summer_playoffs_2017.fillna(7, inplace=True)
summer_playoffs_2017.Place = summer_playoffs_2017.Place.astype(int)
print(summer_playoffs_2017)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0    contractz      4     1       3      25.0   2.50    3.75     8.50  2.93   
1       impact      4     1       3      25.0   1.50    2.75     7.50  3.27   
2       jensen      4     1       3      25.0   5.25    2.00     5.75  5.50   
3     smoothie      4     1       3      25.0   1.50    3.25     7.75  2.85   
4       sneaky      4     1       3      25.0   2.75    2.75     7.00  3.55   
5     aphromoo     11     6       5      54.5   0.45    3.27    10.00  3.19   
6      darshan     11     6       5      54.5   3.91    2.36     5.82  4.12   
7         huhi     11     6       5      54.5   3.82    1.64     7.18  6.72   
8      omargod     11     6       5      54.5   1.09    3.36     9.00  3.00   
9     cody sun      7     4       3      57.1   4.29    2.57     5.86  3.94   
10       flame      7     4       3      57.1   3.14    1.71     7.57  6.25   
11       olleh      7     4       3      57.1   0.71

In [94]:
spring_split_2018 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202018%20Spring&TS%5Bshowstats%5D=2018&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2018%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_split_2018.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2018_Season/Spring_Playoffs')

spring_split_2018 = pd.merge(left=spring_split_2018, right=spring_ranks, on='Team', how='left')
spring_split_2018.fillna(7, inplace=True)
spring_split_2018.Place = spring_split_2018.Place.astype(int)
print(spring_split_2018)


             ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0      aphromoo     19    13       6      68.4   0.47    2.11     6.89  3.50   
1      cody sun     19    13       6      68.4   4.16    1.05     4.47  8.20   
2        meteos     19    13       6      68.4   1.26    1.58     6.42  4.87   
3           ryu     19    13       6      68.4   2.26    1.68     4.68  4.13   
4       ssumday     19    13       6      68.4   1.95    1.74     4.89  3.94   
5        jensen     20    12       8      60.0   3.45    1.20     5.00  7.04   
6      smoothie     20    12       8      60.0   0.60    2.30     7.85  3.67   
7        sneaky     20    12       8      60.0   2.60    1.85     5.25  4.24   
8        apollo     20    11       9      55.0   2.25    1.55     4.20  4.16   
9       febiven     20    11       9      55.0   3.20    0.90     3.35  7.28   
10       hakuho     20    11       9      55.0   0.60    1.60     6.15  4.22   
11         lira     20    11       9    

In [95]:
spring_playoffs_2018 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202018%20Spring%20Playoffs&TS%5Bshowstats%5D=2018&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2018%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2018.info())

spring_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2018_Season/Spring_Playoffs')

spring_playoffs_2018 = pd.merge(left=spring_playoffs_2018, right=spring_ranks, on='Team', how='left')
spring_playoffs_2018.fillna(7, inplace=True)
spring_playoffs_2018.Place = spring_playoffs_2018.Place.astype(int)
print(spring_playoffs_2018)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists    KDA  \
0     aphromoo      8     3       5      37.5   0.13    2.25     4.63   2.11   
1     cody sun      8     3       5      37.5   2.75    1.63     2.88   3.46   
2       meteos      8     3       5      37.5   2.00    2.63     2.75   1.81   
3          ryu      8     3       5      37.5   1.25    1.75     3.75   2.86   
4      ssumday      8     3       5      37.5   1.50    1.63     3.25   2.92   
5       jensen      3     0       3       0.0   1.67    0.67     3.67   8.00   
6     smoothie      3     0       3       0.0   1.00    3.67     3.33   1.18   
7       sneaky      3     0       3       0.0   3.67    0.67     2.67   9.50   
8       apollo     12     5       7      41.7   2.25    1.33     4.17   4.81   
9      febiven     12     5       7      41.7   2.25    1.50     4.08   4.22   
10      hakuho     12     5       7      41.7   0.67    2.08     5.00   2.72   
11        lira     12     5       7     

In [96]:
summer_split_2018 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202018%20Summer&TS%5Bshowstats%5D=2018&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2018%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_split_2018.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2018_Season/Summer_Playoffs')

summer_split_2018 = pd.merge(left=summer_split_2018, right=summer_ranks, on='Team', how='left')
summer_split_2018.fillna(7, inplace=True)
summer_split_2018.Place = summer_split_2018.Place.astype(int)
print(summer_split_2018)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists    KDA  \
0     aphromoo     20    12       8      60.0   0.90    2.75     8.45   3.40   
1     cody sun     20    12       8      60.0   4.25    1.80     5.50   5.42   
2          ryu     20    12       8      60.0   2.75    2.25     6.80   4.24   
3      ssumday     20    12       8      60.0   3.85    1.40     4.80   6.18   
4         anda     14     8       6      57.1   2.29    2.86     7.21   3.33   
5       meteos      6     4       2      66.7   1.33    2.33     6.33   3.29   
6       jensen     13     9       4      69.2   4.31    1.77     6.23   5.96   
7       sneaky     12    10       2      83.3   3.08    2.00     5.83   4.46   
8       blaber      9     8       1      88.9   2.89    2.44     7.44   4.23   
9        keith      4     1       3      25.0   1.50    2.75     3.00   1.64   
10     febiven     18     6      12      33.3   2.17    2.28     4.61   2.98   
11        solo     18     6      12     

In [97]:
summer_playoffs_2018 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=NA%20LCS%202018%20Summer%20Playoffs&TS%5Bshowstats%5D=2018&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=NA%20LCS/2018%20Season/Summer%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(summer_playoffs_2018.info())

summer_ranks = winners15('https://lol.gamepedia.com/NA_LCS/2018_Season/Summer_Playoffs')

summer_playoffs_2018 = pd.merge(left=summer_playoffs_2018, right=summer_ranks, on='Team', how='left')
summer_playoffs_2018.fillna(7, inplace=True)
summer_playoffs_2018.Place = summer_playoffs_2018.Place.astype(int)
print(summer_playoffs_2018)


            ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists    KDA  \
0         anda     12     6       6      50.0   1.50    2.08     5.33   3.28   
1     aphromoo     12     6       6      50.0   0.25    2.92     6.33   2.26   
2          ryu     12     6       6      50.0   2.92    1.92     4.42   3.83   
3      ssumday     12     6       6      50.0   2.58    1.67     4.25   4.10   
4     cody sun      7     4       3      57.1   3.00    1.14     4.43   6.50   
5       rikara      5     2       3      40.0   1.00    1.80     4.80   3.22   
6       sneaky      8     3       5      37.5   3.00    2.88     3.88   2.39   
7       jensen      5     1       4      20.0   1.60    1.80     5.00   3.67   
8       blaber      4     1       3      25.0   2.00    3.25     4.50   2.00   
9      damonte      5     2       3      40.0   3.00    2.80     3.60   2.36   
10     dardoch      5     2       3      40.0   3.00    2.40     5.20   3.42   
11        huni      5     2       3     

In [98]:
spring_split_2019 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=LCS%202019%20Spring&TS%5Bshowstats%5D=2019&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=LCS/2019%20Season/Spring%20Season&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_split_2019.info())

spring_ranks = winners15('https://lol.gamepedia.com/LCS/2019_Season/Spring_Playoffs')

spring_split_2019 = pd.merge(left=spring_split_2019, right=spring_ranks, on='Team', how='left')
spring_split_2019.fillna(7, inplace=True)
spring_split_2019.Place = spring_split_2019.Place.astype(int)
print(spring_split_2019)


              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists  \
0           bang     18     4      14      22.2   2.22    1.39     3.44   
1           anda     16     4      12      25.0   1.13    2.50     3.81   
2       aphromoo     16     4      12      25.0   0.44    2.63     3.81   
3        ssumday     16     4      12      25.0   2.38    1.81     2.44   
4           huhi     12     4       8      33.3   1.92    2.08     3.42   
5         soligo      6     0       6       0.0   1.33    2.83     2.50   
6        fakegod      2     0       2       0.0   0.50    2.50     2.50   
7         fragas      2     0       2       0.0   0.00    5.00     6.00   
8          stunt      2     0       2       0.0   1.00    3.50     4.50   
9         sneaky     17    13       4      76.5   4.12    1.41     5.12   
10         nisqy     16    13       3      81.3   3.69    1.19     6.38   
11        blaber      2     2       0     100.0   3.00    2.50     7.00   
12       diamond      1  

In [99]:
spring_playoffs_2019 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&TS%5Btournament%5D=LCS%202019%20Spring%20Playoffs&TS%5Bshowstats%5D=2019&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=LCS/2019%20Season/Spring%20Playoffs&pfRunQueryFormName=TournamentPlayerInformation')
#check that the second and third .info() prints have the same rows
#print(spring_playoffs_2019.info())

spring_ranks = winners15('https://lol.gamepedia.com/LCS/2019_Season/Spring_Playoffs')

spring_playoffs_2019 = pd.merge(left=spring_playoffs_2019, right=spring_ranks, on='Team', how='left')
spring_playoffs_2019.fillna(7, inplace=True)
spring_playoffs_2019.Place = spring_playoffs_2019.Place.astype(int)
print(spring_playoffs_2019)


              ID  Games  Wins  Losses  Win Rate  Kills  Deaths  Assists   KDA  \
0          nisqy      5     2       3      40.0   2.00    2.00     4.40  3.20   
1         sneaky      5     2       3      40.0   2.00    2.20     4.60  3.00   
2         apollo      4     1       3      25.0   1.75    3.00     2.75  1.50   
3          fenix      4     1       3      25.0   2.00    3.00     3.25  1.75   
4         hakuho      4     1       3      25.0   0.50    3.00     4.50  1.67   
5           rush      4     1       3      25.0   0.50    5.00     5.50  1.20   
6           solo      4     1       3      25.0   2.75    2.75     2.75  2.00   
7           jayj      8     3       5      37.5   0.13    3.50     5.63  1.64   
8       santorin      8     3       5      37.5   1.88    1.75     4.88  3.86   
9      contractz      5     2       3      40.0   2.40    3.20     5.80  2.56   
10        deftly      5     2       3      40.0   3.40    1.40     4.60  5.71   
11       froggen      5     

In [100]:
#Summer playoffs haven't happened yet, commented out until then

#summer_split_2019 = pull_lcs_stats('https://lol.gamepedia.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=LCS%202019%20Summer&TS%5Bpreload%5D=TournamentByPlayer&pfRunQueryFormName=TournamentStatistics', 'https://lol.gamepedia.com/Special:RunQuery/TournamentPlayerInformation?TPI%5Bpage%5D=LCS/2019%20Season/Summer%20Season&pfRunQueryFormName=TournamentPlayerInformation')
##drop a dup
#summer_split_2019 = summer_split_2019.drop(13)
##check that the second and third .info() prints have the same rows
##print(summer_split_2019.info())

#summer_ranks = winners15('https://lol.gamepedia.com/LCS/2019_Season/Summer_Playoffs')

#summer_split_2019 = pd.merge(left=summer_split_2019, right=summer_ranks, on='Team', how='left')
#summer_split_2019.fillna(7, inplace=True)
#summer_split_2019.Place = summer_split_2019.Place.astype(int)
#print(summer_split_2019)




In [101]:
#let's put season tags on them all now
#2013
spring_season_2013 = seasontag(spring_split_2013, 'spring', 'regular season', 2013)
spring_playoffs_2013 = seasontag(spring_playoffs_2013, 'spring', 'playoffs', 2013)
summer_season_2013 = seasontag(summer_split_2013, 'summer', 'regular season', 2013)
summer_playoffs_2013 = seasontag(summer_playoffs_2013, 'summer', 'playoffs', 2013)
#2014
spring_season_2014 = seasontag(spring_split_2014, 'spring', 'regular season', 2014)
spring_playoffs_2014 = seasontag(spring_playoffs_2014, 'spring', 'playoffs', 2014)
summer_season_2014 = seasontag(summer_split_2014, 'summer', 'regular season', 2014)
summer_playoffs_2014 = seasontag(summer_playoffs_2014, 'summer', 'playoffs', 2014)
#2015
spring_season_2015 = seasontag(spring_split_2015, 'spring', 'regular season', 2015)
spring_playoffs_2015 = seasontag(spring_playoffs_2015, 'spring', 'playoffs', 2015)
summer_season_2015 = seasontag(summer_split_2015, 'summer', 'regular season', 2015)
summer_playoffs_2015 = seasontag(summer_playoffs_2015, 'summer', 'playoffs', 2015)
#2016
spring_season_2016 = seasontag(spring_split_2016, 'spring', 'regular season', 2016)
spring_playoffs_2016 = seasontag(spring_playoffs_2016, 'spring', 'playoffs', 2016)
summer_season_2016 = seasontag(summer_split_2016, 'summer', 'regular season', 2016)
summer_playoffs_2016 = seasontag(summer_playoffs_2016, 'summer', 'playoffs', 2016)
#2017
spring_season_2017 = seasontag(spring_split_2017, 'spring', 'regular season', 2017)
spring_playoffs_2017 = seasontag(spring_playoffs_2017, 'spring', 'playoffs', 2017)
summer_season_2017 = seasontag(summer_split_2017, 'summer', 'regular season', 2017)
summer_playoffs_2017 = seasontag(summer_playoffs_2017, 'summer', 'playoffs', 2017)
#2018
spring_season_2018 = seasontag(spring_split_2018, 'spring', 'regular season', 2018)
spring_playoffs_2018 = seasontag(spring_playoffs_2018, 'spring', 'playoffs', 2018)
summer_season_2018 = seasontag(summer_split_2018, 'summer', 'regular season', 2018)
summer_playoffs_2018 = seasontag(summer_playoffs_2018, 'summer', 'playoffs', 2018)
#2019
spring_season_2019 = seasontag(spring_split_2019, 'spring', 'regular season', 2019)
spring_playoffs_2019 = seasontag(spring_playoffs_2019, 'spring', 'playoffs', 2019)
#summer_season_2019 = seasontag(summer_split_2015, 'summer', 'regular season', 2019)
#summer_playoffs_2019 = seasontag(summer_playoffs_2015, 'summer', 'playoffs', 2019)

In [102]:
#Final step, merge them all by rows into one big dataframe
LCSdata = pd.concat([spring_season_2013, spring_playoffs_2013, summer_season_2013, summer_playoffs_2013,
                     spring_season_2014, spring_playoffs_2014, summer_season_2014, summer_playoffs_2014,
                     spring_season_2015, spring_playoffs_2015, summer_season_2015, summer_playoffs_2015,
                     spring_season_2016, spring_playoffs_2016, summer_season_2016, summer_playoffs_2016,
                     spring_season_2017, spring_playoffs_2017, summer_season_2017, summer_playoffs_2017,
                     spring_season_2018, spring_playoffs_2018, summer_season_2018, summer_playoffs_2018,
                     spring_season_2019, spring_playoffs_2019],
                   sort=False)
print(LCSdata.info)

<bound method DataFrame.info of     Year   Split            Kind               Team            ID  \
0   2013  spring  regular season  compLexity Gaming        brunch   
1   2013  spring  regular season  compLexity Gaming   lautemortis   
2   2013  spring  regular season  compLexity Gaming         meyea   
3   2013  spring  regular season  compLexity Gaming        nickwu   
4   2013  spring  regular season  compLexity Gaming       chuuper   
..   ...     ...             ...                ...           ...   
19  2019  spring        playoffs       Team SoloMid      akaadian   
20  2019  spring        playoffs       Team SoloMid      bjergsen   
21  2019  spring        playoffs       Team SoloMid  broken blade   
22  2019  spring        playoffs       Team SoloMid      smoothie   
23  2019  spring        playoffs       Team SoloMid          zven   

                    Name  Place  Games  Wins  Losses  ...  Assists   KDA  \
0          Josh Abrantes      7     28     9      19  ...     6

In [109]:
print(LCSdata.info())

LCSdata.to_csv('LCSdata13_19sp.csv', header=True, index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034 entries, 0 to 23
Data columns (total 23 columns):
Year               1034 non-null int64
Split              1034 non-null object
Kind               1034 non-null object
Team               1034 non-null object
ID                 1034 non-null object
Name               1034 non-null object
Place              1034 non-null int64
Games              1034 non-null int64
Wins               1034 non-null int64
Losses             1034 non-null int64
Win Rate           1034 non-null float64
Kills              1034 non-null float64
Deaths             1034 non-null float64
Assists            1034 non-null float64
KDA                1034 non-null float64
CS                 1034 non-null float64
CS per Minute      1034 non-null float64
Gold               1014 non-null float64
Gold per Minute    1014 non-null float64
Kill Part.         886 non-null float64
Kill Share         886 non-null float64
Gold Share         886 non-null float64
Champs Play